# Creamos Base de datos #

In [45]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE IF NOT EXISTS Project_1")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: CREATE DATABASE IF NOT EXISTS Project_1


# Creamos tablas #

In [46]:


# Creamos una funcion para crear la tabla correspondiente a cada extension de archivo, dentro de la Base de Datos Project_1

def creacion_tablas (query_tablas):
    
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='Project_1')


    mycursor = cnx.cursor()
    try:
        mycursor.execute(query_tablas)
        cnx.commit()
        print(mycursor)
    
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)



### Definimos query de creacion de tablas sql:

In [47]:
tabla_sql = ('''CREATE TABLE IF NOT EXISTS data_sql (
index_sql INT NOT NULL PRIMARY KEY , 
q10_part_1 VARCHAR (55), 
q10_part_2 VARCHAR (55), 
q10_part_3 VARCHAR (55), 
q10_part_4 VARCHAR (55), 
q10_part_5 VARCHAR (55), 
q10_part_6 VARCHAR (55), 
q10_part_7 VARCHAR (55), 
q10_part_8 VARCHAR (55), 
q10_part_9 VARCHAR (55), 
q10_part_10 VARCHAR (55), 
q10_part_11 VARCHAR (55), 
q10_part_12 VARCHAR (55), 
q10_part_13 VARCHAR (55), 
q10_part_14 VARCHAR (55), 
q10_part_15 VARCHAR (55),
q10_part_16 VARCHAR (55), 
q10_other VARCHAR (55)
);''')

In [48]:
tabla_xml = ('''CREATE TABLE IF NOT EXISTS data_xml (
`index_xml` INT NOT NULL PRIMARY KEY,
`time` TEXT (50),
`age` TEXT (50),
`gender` TEXT (50),
`index_sql` INT,
CONSTRAINT `fk_data_xml_data_sql`
FOREIGN KEY (`index_sql`)
REFERENCES `data_sql` (`index_sql`)
ON DELETE CASCADE
ON UPDATE CASCADE
)''')

In [49]:
tabla_txt = (''' CREATE TABLE IF NOT EXISTS data_txt (
index_txt INT NOT NULL PRIMARY KEY,
index_sql INT,
q3 TEXT(150),
q4 TEXT(150),
q5 TEXT(150),
q6 TEXT(150),
q7 TEXT(150),
q8 TEXT(150),
q9 TEXT(150),
q11 TEXT(150),
q12 TEXT(150),
q13 TEXT(150),
q14 TEXT(150),
q15 TEXT(150),
q16 TEXT(150),
q17 TEXT(150),
q20 TEXT(150),
q21 TEXT(150),
q22 TEXT(150),
q23 TEXT(150),
q24 TEXT(150),
q25 TEXT(150),
q26 TEXT(150),
q32 TEXT(150),
q33 TEXT(150),
q34 TEXT(150),
q35 TEXT(150),
q41 TEXT(150),
CONSTRAINT `fk_data_txt_data_sql`
    FOREIGN KEY (index_sql)
    REFERENCES `data_sql` (index_sql)
    ON DELETE CASCADE
    ON UPDATE CASCADE
    );''')

### Corremos las funciones para que se creen cada una de las tablas.

In [50]:

creacion_tablas(tabla_sql)


MySQLCursor: CREATE TABLE IF NOT EXISTS data_sql (
in..


In [51]:
creacion_tablas(tabla_xml)


MySQLCursor: CREATE TABLE IF NOT EXISTS data_xml (
`i..


In [52]:
creacion_tablas (tabla_txt)

MySQLCursor:  CREATE TABLE IF NOT EXISTS data_txt (
i..


Una vez creada la base de datos y la tabla tenemos que insertar los valores que tenemos para cada una de ellas, se nos presenta la situacion de que los datos que contiene cada archivo no estan  listos para ser cargados, es necesario que realicemos limpieza de los mismos con distintas funciones dependiendo del tipo de archivo que tratemos, para ellos vamos a definir una clase que identifique que extencion de archivo estamos leyendo y proceda a realizarle las modificaciones necesarias para cargar sus datos en la tabla correspondiente. 

#### Importamos librerias:

In [53]:
import re
import os
import os.path
import xml.etree.ElementTree as ET
import mysql.connector


In [54]:
class Datos_limpios:

    def __init__ (self,nombre_archivo, extension_archivo):

        self.nombre_archivo = nombre_archivo
        self.extension_archivo = extension_archivo

     

    def  abrir_archivos (self):
    
            #nombre = os.path.splitext(self)[0]#
            #print(nombre)
            #extension_archivo = os.path.splitext(self)[1]
            #print(extension)

            if  self.extension_archivo == '.xml':  #funciones xml para lectura ficheros.
                
                tree = ET.parse(self.nombre_archivo)
                root = tree.getroot()

                self.nombre_archivo = []
                for child in root:
                        xml_dict = {}
                        for subchild in child:
                            xml_dict[subchild.tag] = subchild.text
                        self.nombre_archivo.append(xml_dict)
                
                return self.nombre_archivo

                      
            elif self.extension_archivo == '.txt':  #funciones txt para lectura ficheros.

                with open(self.nombre_archivo,'r') as file:
                    self.nombre_archivo = file.readlines()

                return self.nombre_archivo

           

            #elif extension == '.sql':
            #funciones sql para lectura ficheros(ver que ya lo leemos directamente en sql!)
    
    def limpieza_datos (self,lista_lectura, extension_archivo):

           
            #nombre = os.path.splitext(self)[0]
            #extension = os.path.splitext(self)[1]

        if self.extension_archivo == '.xml':
               
            #def limpieza_datos (self):
        #nombre = os.path.splitext(self)[0]
        #extension = os.path.splitext(self)[1]
                for dictionary in self.nombre_archivo:
                    if dictionary.get('gender') == '0':
                        dictionary['gender'] = 'Man'
                    elif dictionary.get('gender') == '1':
                        dictionary['gender'] = 'Woman'
                    elif dictionary.get('gender') == '2':
                        dictionary['gender'] = 'Nonbinary'
                    elif dictionary.get('gender') == '3':
                        dictionary['gender'] = 'Prefer not to say'
                    elif dictionary.get('gender') == '4':
                        dictionary['gender'] = 'Prefer to self-describe'
                    else:    
                        print(f"There is an error, the unlisted values is : {dictionary.get('gender')}")

                    dictionary.pop("level_0")
            
                lista_tupla_xml= []
                for i in self.nombre_archivo:
                    lista_tupla_xml.append(tuple(i.values()))

                return lista_tupla_xml
   
            
        elif self.extension_archivo == '.txt':
                
                lista_puntocoma = []
                lista_espacios = []
                lista_under = []  
                lista_final = []  
                lista_tupla = []
                lista_tupla_txt = []

            #def limpieza_txt (self): # Vamos a quitar los puntos y comas de la lista y convertir cada string en lista.
                for i in self.nombre_archivo:
            
                    lista_intermedia = i.split(";")
                    lista_puntocoma.append(lista_intermedia)

            #def espacios(self):# Vamos a quitar los espacios de mas en cada string.
                for sublista in lista_puntocoma[1:]:

                    lista_almancenado_strings = []

                    for palabra in sublista:
                        palabra = palabra.strip()
                        lista_almancenado_strings.append(palabra)
                
                    lista_espacios.append(lista_almancenado_strings)

             #def under(self):
                for sublista in lista_espacios[:]:
                    
                    lista_almancenado_under = []

                    for palabra in sublista:
                        i = palabra.replace('<','under')
                        lista_almancenado_under.append(i)
        
                    lista_under.append(lista_almancenado_under)

            #def null(self):
                for sublista in lista_under[:]:

                    lista_almancenado_null = []
                    for palabra in sublista:
                        x = palabra.replace('null','NULL')
                        lista_almancenado_null.append(x)
    
                    lista_final.append(lista_almancenado_null)

                for z in lista_final:
                    lista_tupla = tuple(z) 
                    lista_tupla_txt.append(lista_tupla)   

                return lista_tupla_txt
          
            #elif extension == '.sql':


            #funciones sql limpieza


Creamos cada una de las clases por cada extension de archivo.


In [55]:
xml_class = Datos_limpios('data_xml.xml', '.xml')

In [56]:
txt_class = Datos_limpios('data_txt.txt', '.txt')

In [57]:
sql_class = Datos_limpios('data_sql.sql', '.sql')

Llamamos a las funciones de la clase de apertura y limpieza del archivo.


In [58]:
xml_lista = xml_class.abrir_archivos()


In [59]:
xml_limpio = xml_class.limpieza_datos(xml_lista, '.xml')

In [60]:
xml_limpio [25:50]

[('26', '7750', '45-49', 'Man'),
 ('27', '607', '22-24', 'Man'),
 ('28', '525', '22-24', 'Woman'),
 ('29', '501', '18-21', 'Man'),
 ('30', '415', '22-24', 'Man'),
 ('31', '317', '30-34', 'Man'),
 ('32', '171', '22-24', 'Nonbinary'),
 ('33', '744', '30-34', 'Woman'),
 ('34', '668', '22-24', 'Man'),
 ('35', '460', '30-34', 'Woman'),
 ('36', '244', '25-29', 'Woman'),
 ('37', '901', '30-34', 'Man'),
 ('38', '625', '22-24', 'Man'),
 ('39', '1239', '18-21', 'Man'),
 ('40', '229993', '25-29', 'Man'),
 ('41', '312', '25-29', 'Man'),
 ('42', '976', '55-59', 'Man'),
 ('43', '1890', '30-34', 'Woman'),
 ('44', '357', '18-21', 'Man'),
 ('45', '660', '40-44', 'Man'),
 ('46', '1524', '22-24', 'Man'),
 ('47', '453', '18-21', 'Woman'),
 ('48', '793', '25-29', 'Man'),
 ('49', '557', '30-34', 'Woman'),
 ('50', '5215', '25-29', 'Man')]

In [61]:
txt_lista = txt_class.abrir_archivos()

In [62]:
txt_limpio = txt_class.limpieza_datos(txt_lista, '.txt')

In [63]:
#sql_lista = sql_class.lectura_archivo()

In [64]:
#sql_limpio = sql_class.limpieza_datos(sql_lista)

Una vez limpios los datos necesitamos extraer los valores de cada archivo en una query para poder asi cargarlos en las tablas respectivas.


In [65]:
query_val_xml = xml_limpio


In [66]:
query_val_txt = txt_limpio

In [67]:
#uery_val_sql = sql_limpio

In [68]:
xml_limpio[0]

('1', '784', '50-54', 'Man')

Para la carga de datos necesitamos tener los nombres de columnas de cada una de las tablas, por lo que creamos la query para cada una de las tablas:

In [69]:
query_sql_xml = '''INSERT INTO data_xml (index_xml, time, age, gender) VALUES (%s, %s,%s, %s)'''

In [76]:
query_sql_txt ='''INSERT INTO data_txt (index_txt, q3, q4, q5, q6, q7, q8, q9, q11, q12, q13, q14, q15, q16, q17, q20, q21, q22, q23, q24, q25, q26, q32, q33, q34, q35, q41)VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) '''

In [71]:
query_sql_sql = '''INSERT INTO data_sql (`index_sql`,`q10_part_1`,`q10_part_2`,`q10_part_3`, `q10_part_4`, `q10_part_5`, `q10_part_6`,`q10_part_7`, `q10_part_8`,`q10_part_9`,`q10_part_10`, `q10_part_11`, `q10_part_12`, `q10_part_13`, `q10_part_14`, `q10_part_15`, `q10_part_16`, `q10_other`)'''

 # Insertamos datos en las tablas #

In [77]:
def insertar_datos (query_sql,query_values):

  cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='Project_1')


  mycursor = cnx.cursor()
  #sql = query_sql
  #val = query_values
  
  try: 
      #mycursor.execute('SET FOREING_KEY_CHECKS = 0')
      mycursor.executemany(query_sql, query_values)
      cnx.commit()
      print(mycursor.rowcount, "registro/s insertado/s.")
      #mycursor.execute('SET FOREING_KEY_CHECKS = 1')

  except mysql.connector.Error as err:
      print(err)
      print("Error Code:", err.errno)
      print("SQLSTATE", err.sqlstate)
      print("Message", err.msg)
    
  else:
      mycursor.close()
      cnx.close()

In [78]:
insertar_datos(query_sql_xml, query_val_xml)

1062 (23000): Duplicate entry '1' for key 'data_xml.PRIMARY'
Error Code: 1062
SQLSTATE 23000
Message Duplicate entry '1' for key 'data_xml.PRIMARY'


In [79]:
insertar_datos(query_sql_txt, query_val_txt)

25972 registro/s insertado/s.


In [80]:
insertar_datos(query_sql_sql, query_val_sql)

NameError: name 'query_val_sql' is not defined